<a href="https://colab.research.google.com/github/svondracek0/Data-Science-Practicum/blob/main/Vondracek_Housing_Prices_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Housing Prices Regresison
*Stepan Vondracek, 11/12/2021*



In this task, I was supposed to conduct an exploratory data analysis over the *housing price dataset*.

At first, I mount my Google Drive to  access the data.

In [43]:
% cd "/content/drive/MyDrive/"
% mkdir "intern_data"
% ls

/content/drive/MyDrive
mkdir: cannot create directory ‘intern_data’: File exists
 Bayes_exc04_a
'Colab Notebooks'/
 dataset2/
 hw/
 intern_data/
 Intern_task_data.7z
'Kopie AII 1 Autumn 2021 CLASS NOTES Gr 2.gdoc'
'Osobní dotazník HF-30579. docx.docx'
'Osobní dotazník HF-30579. docx.gdoc'
'Štěpán úvodní video.gdoc'
 trump_speech.txt
 Vondracek-Diplom.pdf
 Vondracek_Housing-Prices-Task.ipynb
 Vondracek_hw09_dspract.ipynb
 Vondracek_presentation.gslides


In [45]:
!pip install pyunpack
!pip install patool
from pyunpack import Archive
Archive("Intern_task_data.7z").extractall("./intern_data") #hereby I extract the .7z file in my Drive


Before the actual analysis, I find it useful to provide some reasoning by assessing the description of the data

Our datasets contain the following variables:
+ 'MSSubClass' - categorial variable describing the cateory of a house
+ 'MSZonong' - categorical variable, I expect this variable to have significant impact
+ 'LotFrontage' - continuous variable, also expected significant postitive effect
+ 'Street' - binary variable, paved access expected to have positive effect
+  'Alley' - probably less significant than street, similar sign of the effect
+  'LotShape' - categorical, the more regular, the higher the price
+ 'Landcontour' - probably the fatter the higher the price
+ 'Utilities' - categorical, probably of very high importance
+ '

In [50]:
desc_file = "data_description.txt"
file_desc = open(desc_file, "r")
print(file_desc.read())

MSSubClass: Identifies the type of dwelling involved in the sale.	

        20	1-STORY 1946 & NEWER ALL STYLES
        30	1-STORY 1945 & OLDER
        40	1-STORY W/FINISHED ATTIC ALL AGES
        45	1-1/2 STORY - UNFINISHED ALL AGES
        50	1-1/2 STORY FINISHED ALL AGES
        60	2-STORY 1946 & NEWER
        70	2-STORY 1945 & OLDER
        75	2-1/2 STORY ALL AGES
        80	SPLIT OR MULTI-LEVEL
        85	SPLIT FOYER
        90	DUPLEX - ALL STYLES AND AGES
       120	1-STORY PUD (Planned Unit Development) - 1946 & NEWER
       150	1-1/2 STORY PUD - ALL AGES
       160	2-STORY PUD - 1946 & NEWER
       180	PUD - MULTILEVEL - INCL SPLIT LEV/FOYER
       190	2 FAMILY CONVERSION - ALL STYLES AND AGES

MSZoning: Identifies the general zoning classification of the sale.
		
       A	Agriculture
       C	Commercial
       FV	Floating Village Residential
       I	Industrial
       RH	Residential High Density
       RL	Residential Low Density
       RP	Residential Low Density Park 
       RM

In [57]:
import pandas as pd
data_1 = pd.read_csv("data_1.csv")
data_2 = pd.read_csv("data_2.csv")

for elem in [data_1, data_2]:
  print(elem.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1149 non-null   int64  
 1   MSSubClass     1149 non-null   int64  
 2   MSZoning       1149 non-null   object 
 3   LotFrontage    947 non-null    float64
 4   LotArea        1149 non-null   int64  
 5   Street         1149 non-null   object 
 6   Alley          70 non-null     object 
 7   LotShape       1149 non-null   object 
 8   LandContour    1149 non-null   object 
 9   Utilities      1149 non-null   object 
 10  LotConfig      1149 non-null   object 
 11  LandSlope      1149 non-null   object 
 12  Neighborhood   1149 non-null   object 
 13  Condition1     1149 non-null   object 
 14  Condition2     1149 non-null   object 
 15  BldgType       1149 non-null   object 
 16  HouseStyle     1149 non-null   object 
 17  OverallQual    1149 non-null   int64  
 18  OverallC

In [74]:
for var in data_1:
  if var not in data_2:
    print(var)
# these 3 variables are not present in the second data set, data_1 contains all of data_2 variables

Alley
FireplaceQu
PoolQC


The two datasets seem to be independent in terms of observations. The first dataset contains more variables as well as more observations. It is further convenient to combine the two datasets. The former data_2 rows will naturally contain NAs for the three variables. Talking about the NAs, it is usefull to find out how are the NAs handled in the dataset. I can do it by searching the .txt file - NAs are apparently marked as 'NA'.

In [81]:
dfs = [data_1, data_2]

data_merged = pd.concat(dfs, join="outer") #hereby I merge the two datasets, the outer join ensures that all variables will be kept
len(pd.unique(data_merged["Id"])) == len(data_merged) #the length of unique values equals the n of rows

True

The following function prints a table of missong or zero vaues in our data set. From naively assessing these variables, we may conclude that variables, which are the most affected by NAs, are most likel the ones of little significance.

In [90]:
def missing_zero_values_table(df):
        zero_val = (df == 0.00).astype(int).sum(axis=0)
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mz_table = pd.concat([zero_val, mis_val, mis_val_percent], axis=1)
        mz_table = mz_table.rename(
        columns = {0 : 'Zero Values', 1 : 'Missing Values', 2 : '% of Total Values'})
        mz_table['Total Zero Missing Values'] = mz_table['Zero Values'] + mz_table['Missing Values']
        mz_table['% Total Zero Missing Values'] = 100 * mz_table['Total Zero Missing Values'] / len(df)
        mz_table['Data Type'] = df.dtypes
        mz_table = mz_table[
            mz_table.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns and " + str(df.shape[0]) + " Rows.\n"      
            "There are " + str(mz_table.shape[0]) +
              " columns that have missing values.")
        return mz_table

missing_zero_values_table(data_merged)

Your selected dataframe has 81 columns and 1460 Rows.
There are 19 columns that have missing values.


,Zero Values,Missing Values,% of Total Values,Total Zero Missing Values,% Total Zero Missing Values,Data Type
PoolQC,0,1458,99.9,1458,99.9,object
MiscFeature,0,1406,96.3,1406,96.3,object
Alley,0,1390,95.2,1390,95.2,object
Fence,0,1179,80.8,1179,80.8,object
FireplaceQu,0,856,58.6,856,58.6,object
LotFrontage,0,259,17.7,259,17.7,float64
GarageType,0,81,5.5,81,5.5,object
GarageYrBlt,0,81,5.5,81,5.5,float64
GarageFinish,0,81,5.5,81,5.5,object
GarageQual,0,81,5.5,81,5.5,object


In [92]:
data_merged.corr().style.background_gradient(cmap='coolwarm')

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,1.000000,0.011156,-0.010601,-0.033226,-0.028365,0.012609,-0.012713,-0.021998,-0.050298,-0.005024,-0.005968,-0.007940,-0.015415,0.010496,0.005590,-0.044230,0.008273,0.002289,-0.020155,0.005587,0.006784,0.037719,0.002951,0.027239,-0.019772,0.000072,0.016570,0.017634,-0.029643,-0.000477,0.002889,-0.046635,0.001330,0.057044,-0.006242,0.021172,0.000712,-0.021917
MSSubClass,0.011156,1.000000,-0.386347,-0.139781,0.032628,-0.059316,0.027850,0.040581,0.022936,-0.069836,-0.065649,-0.140759,-0.238518,-0.251758,0.307886,0.046474,0.074853,0.003491,-0.002333,0.131608,0.177354,-0.023438,0.281721,0.040380,-0.045569,0.085072,-0.040110,-0.098672,-0.012579,-0.006100,-0.012037,-0.043825,-0.026030,0.008283,-0.007683,-0.013585,-0.021407,-0.084284
LotFrontage,-0.010601,-0.386347,1.000000,0.426095,0.251646,-0.059213,0.123349,0.088866,0.193458,0.233633,0.049900,0.132644,0.392075,0.457181,0.080177,0.038469,0.402797,0.100949,-0.007234,0.198769,0.053532,0.263170,-0.006069,0.352096,0.266639,0.070250,0.285691,0.344997,0.088521,0.151972,0.010700,0.070029,0.041383,0.206167,0.003368,0.011200,0.007450,0.351799
LotArea,-0.033226,-0.139781,0.426095,1.000000,0.105806,-0.005636,0.014228,0.013788,0.104160,0.214103,0.111170,-0.002618,0.260833,0.299475,0.050986,0.004779,0.263116,0.158155,0.048046,0.126031,0.014259,0.119690,-0.017784,0.190015,0.271364,-0.024947,0.154871,0.180403,0.171698,0.084774,-0.018340,0.020423,0.043160,0.077672,0.038068,0.001205,-0.014261,0.263843
OverallQual,-0.028365,0.032628,0.251646,0.105806,1.000000,-0.091932,0.572323,0.550684,0.411876,0.239666,-0.059119,0.308159,0.537808,0.476224,0.295493,-0.030429,0.593007,0.111098,-0.040150,0.550600,0.273458,0.101676,-0.183882,0.427452,0.396765,0.547766,0.600671,0.562022,0.238923,0.308819,-0.113937,0.030371,0.064886,0.065166,-0.031406,0.070815,-0.027347,0.790982
OverallCond,0.012609,-0.059316,-0.059213,-0.005636,-0.091932,1.000000,-0.375983,0.073741,-0.128101,-0.046231,0.040229,-0.136841,-0.171098,-0.144203,0.028942,0.025494,-0.079686,-0.054942,0.117821,-0.194149,-0.060769,0.012980,-0.087001,-0.057583,-0.023820,-0.324297,-0.185758,-0.151521,-0.003334,-0.032589,0.070356,0.025504,0.054811,-0.001985,0.068777,-0.003511,0.043950,-0.077856
YearBuilt,-0.012713,0.027850,0.123349,0.014228,0.572323,-0.375983,1.000000,0.592855,0.315707,0.249503,-0.049107,0.149040,0.391452,0.281986,0.010308,-0.183784,0.199010,0.187599,-0.038162,0.468271,0.242656,-0.070651,-0.174800,0.095589,0.147716,0.825667,0.537850,0.478954,0.224880,0.188686,-0.387268,0.031355,-0.050364,0.004950,-0.034383,0.012398,-0.013618,0.522897
YearRemodAdd,-0.021998,0.040581,0.088866,0.013788,0.550684,0.073741,0.592855,1.000000,0.179618,0.128451,-0.067759,0.181133,0.291066,0.240379,0.140024,-0.062419,0.287389,0.119470,-0.012337,0.439046,0.183331,-0.040581,-0.149598,0.191740,0.112581,0.642277,0.420622,0.371600,0.205726,0.226298,-0.193919,0.045286,-0.038740,0.005829,-0.010286,0.021490,0.035743,0.507101
MasVnrArea,-0.050298,0.022936,0.193458,0.104160,0.411876,-0.128101,0.315707,0.179618,1.000000,0.264736,-0.072319,0.114442,0.363936,0.344501,0.174561,-0.069071,0.390857,0.085310,0.026673,0.276833,0.201444,0.102821,-0.037610,0.280682,0.249070,0.252691,0.364204,0.373066,0.159718,0.125703,-0.110204,0.018796,0.061466,0.011723,-0.029815,-0.005965,-0.008201,0.477493
BsmtFinSF1,-0.005024,-0.069836,0.233633,0.214103,0.239666,-0.046231,0.249503,0.128451,0.264736,1.000000,-0.050117,-0.495251,0.522396,0.445863,-0.137079,-0.064503,0.208171,0.649212,0.067418,0.058543,0.004262,-0.107355,-0.081007,0.044316,0.260011,0.153484,0.224054,0.296970,0.204306,0.111761,-0.102303,0.026451